In [1]:
#Setup
import pandas as pd
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<p style="font-family: Montserrat; color: #203443">- How can we create a LUSID webhook notification using the Notifications API via the Python SDK?</p>

<p style="font-family: Montserrat; color: #203443">- What are our options for content type, authentication etc. when sending webhook notifications?</p>

<p style="font-family: Montserrat; color: #203443">- How can we create a 3rd party webhook notification using the Notifications API via the Python SDK?</p>

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Creating 3rd Party Webhook Notification - Setup</p>

In [2]:
import lusid_configuration
from lusidjam import RefreshingToken
from fbnsdkutilities import ApiClientFactory

api_factory_configuration = ApiClientFactory(lusid_configuration, token=RefreshingToken())
configuration_sets_api = api_factory_configuration.build(lusid_configuration.api.ConfigurationSetsApi)

configuration_sets_api.generate_access_token()

{'description': 'An access token representing the current user',
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://demo.lusid.com/app/insights/logs/0HMP1FHSF3FNG:00000002',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'ref': 'config://personal/me/00u89xerfkS08fj6e2p7',
 'type': 'Secret',
 'value': '**********'}

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Creating a LUSID Webhook Notification</p>

In [3]:
import pprint

import lusid_notifications
from lusidjam import RefreshingToken
from fbnsdkutilities import ApiClientFactory

api_factory_notifications = ApiClientFactory(lusid_notifications, token=RefreshingToken())
notifications_api = api_factory_notifications.build(lusid_notifications.api.NotificationsApi)
subscriptions_api = api_factory_notifications.build(lusid_notifications.api.SubscriptionsApi)

subscription_scope = "FinbourneUniversity"
subscription_code = "AnyPortfolioCreatedv2"

In [4]:
try:
    subscriptions_api.get_subscription(
        scope=subscription_scope,
        code=subscription_code)
    
    subscriptions_api.delete_subscription(
        scope=subscription_scope,
        code=subscription_code
    )
    
except lusid_notifications.ApiException as api_exception:
    if api_exception.status == 404:
        pass
    else:
        raise

In [5]:
subscriptions_api.create_subscription(
    create_subscription=lusid_notifications.CreateSubscription(
        id=lusid_notifications.ResourceId(
            scope=subscription_scope,
            code=subscription_code,
        ),
        display_name="Any Portfolio Created",
        description="Listens to all TransactionPortfolioCreated events",
        status="Active",
        matching_pattern=lusid_notifications.MatchingPattern(
            event_type="TransactionPortfolioCreated",
            filter="PortfolioScope eq 'FinbourneUniversity' and PortfolioCode startsWith 'NewTradingPortfolio'"
        )
    )
)

{'created_at': datetime.datetime(2023, 3, 13, 3, 33, 1, 127804, tzinfo=tzlocal()),
 'created_by': '00u89xerfkS08fj6e2p7',
 'description': 'Listens to all TransactionPortfolioCreated events',
 'display_name': 'Any Portfolio Created',
 'id': {'code': 'AnyPortfolioCreatedv2', 'scope': 'FinbourneUniversity'},
 'last_modified_at': datetime.datetime(2023, 3, 13, 3, 33, 1, 127804, tzinfo=tzlocal()),
 'last_modified_by': '00u89xerfkS08fj6e2p7',
 'matching_pattern': {'event_type': 'TransactionPortfolioCreated',
                      'filter': "PortfolioScope eq 'FinbourneUniversity' and "
                                'PortfolioCode startsWith '
                                "'NewTradingPortfolio'"},
 'status': 'Active'}

In [6]:
notifications_api.create_webhook_notification(
    scope=subscription_scope,
    code=subscription_code,
    create_webhook_notification=lusid_notifications.CreateWebhookNotification(
        description="New Portfolio Created",
        http_method="POST",
        url="/api/api/transactionportfolios/{{PortfolioScope}}/{{PortfolioCode}}/transactions",
        authentication_type="Lusid",
        content_type="Json",
        content=[
                    {
                        "transactionId": "SeedFundsTransactionUSD",
                        "type": "FundsIn",
                        "instrumentIdentifiers": {
                          "Instrument/default/Currency": "USD",
                        },
                        "transactionDate": "2023-02-01T00:00:00Z",
                        "settlementDate": "2023-02-01T00:00:00Z",
                        "units": 10000000,
                        "transactionPrice": {
                          "price": 1,
                          "type": "Price"
                        },
                        "totalConsideration": {
                          "amount": 10000000,
                          "currency": "USD"
                        },
                        "transactionCurrency": "USD"
                    },
                    {
                        "transactionId": "SeedFundsTransactionGBP",
                        "type": "FundsIn",
                        "instrumentIdentifiers": {
                          "Instrument/default/Currency": "GBP",
                        },
                        "transactionDate": "2023-02-01T00:00:00Z",
                        "settlementDate": "2023-02-01T00:00:00Z",
                        "units": 7500000,
                        "transactionPrice": {
                          "price": 1,
                          "type": "Price"
                        },
                        "totalConsideration": {
                          "amount": 7500000,
                          "currency": "GBP"
                        },
                        "transactionCurrency": "GBP"
                    },
                    {
                        "transactionId": "SeedFundsTransactionAUD",
                        "type": "FundsIn",
                        "instrumentIdentifiers": {
                          "Instrument/default/Currency": "AUD",
                        },
                        "transactionDate": "2023-02-01T00:00:00Z",
                        "settlementDate": "2023-02-01T00:00:00Z",
                        "units": 25000000,
                        "transactionPrice": {
                          "price": 1,
                          "type": "Price"
                        },
                        "totalConsideration": {
                          "amount": 25000000,
                          "currency": "AUD"
                        },
                        "transactionCurrency": "AUD"
                    }
                ]
    )
)

{'content': {'Content': [{'instrumentIdentifiers': {'Instrument/default/Currency': 'USD'},
                          'settlementDate': '2023-02-01T00:00:00.0000000Z',
                          'totalConsideration': {'amount': 10000000,
                                                 'currency': 'USD'},
                          'transactionCurrency': 'USD',
                          'transactionDate': '2023-02-01T00:00:00.0000000Z',
                          'transactionId': 'SeedFundsTransactionUSD',
                          'transactionPrice': {'price': 1, 'type': 'Price'},
                          'type': 'FundsIn',
                          'units': 10000000},
                         {'instrumentIdentifiers': {'Instrument/default/Currency': 'GBP'},
                          'settlementDate': '2023-02-01T00:00:00.0000000Z',
                          'totalConsideration': {'amount': 7500000,
                                                 'currency': 'GBP'},
                    

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Webhook Options</p>

<p style="font-family: Montserrat; color: #203443">Content Types</p>
<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">JSON (application/json)</p></li>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">PlainText (text/plain)</p></li>
</ul>

<p style="font-family: Montserrat; color: #203443">Methods</p>
<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">POST</p></li>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">PUT</p></li>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">DELETE</p></li>
</ul>

<p style="font-family: Montserrat; color: #203443">Authentication Options</p>
<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">Lusid</p></li>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">Bearer</p></li>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">Basic Auth</p></li>
</ul>

In [7]:
try:
    configuration_sets_api.get_configuration_set(
        type="Shared",
        scope="FinbourneUniversity",
        code="jira"
    )
    
    configuration_sets_api.delete_configuration_set(
        type="Shared",
        scope="FinbourneUniversity",
        code="jira"
    )
    
except lusid_configuration.ApiException as api_exception:
    if api_exception.status == 404:
        pass
    else:
        raise

In [8]:
config_response = configuration_sets_api.create_configuration_set(
    create_configuration_set=lusid_configuration.CreateConfigurationSet(
        id=lusid_configuration.ResourceId(
            scope="FinbourneUniversity",
            code="jira"
        ),
        type="Shared",
        description="Access tokens for JIRA"
    )
)

configuration_sets_api.add_configuration_to_set(
    type="Shared",
    scope="FinbourneUniversity",
    code="jira",
    create_configuration_item=lusid_configuration.CreateConfigurationItem(
        key="api-token",
        value="SampleToken",
        value_type="text",
        is_secret=True,
        description="API Key for JIRA"
    )
)

configuration_sets_api.add_configuration_to_set(
    type="Shared",
    scope="FinbourneUniversity",
    code="jira",
    create_configuration_item=lusid_configuration.CreateConfigurationItem(
        key="username",
        value="SampleUsername",
        value_type="text",
        is_secret=False,
        description="Username for JIRA"
    )
)

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Creating 3rd Party Webhook Notification</p>

In [9]:
notifications_api.create_webhook_notification(
    scope=subscription_scope,
    code=subscription_code,
    create_webhook_notification=lusid_notifications.CreateWebhookNotification(
        description="New Portfolio Created - Ticket",
        http_method="POST",
        url="https://acmecorp.atlassian.net/rest/api/3/issue",
        authentication_type="BasicAuth",
        authentication_configuration_item_paths={
            "username": "config://shared/FinbourneUniversity/jira/username",
            "password": "config://shared/FinbourneUniversity/jira/api-token"
        },
        content_type="Json",
        content={
            "fields": {
                "project": {
                    "key": "DEMO"
                },
                "issuetype": {
                    "name": "Task"
                },
                "summary": "TransactionPortfolio {{PortfolioDisplayName}} Created!",
                "description": {
                    "type": "doc",
                    "version": 1,
                    "content": [
                        {
                        "type": "paragraph",
                        "content": [
                            {
                            "text": "A portfolio has been created. Please make your first trades!",
                            "type": "text"
                            }
                        ]
                        },
                        {
                        "type": "paragraph",
                        "content": [
                            {
                            "text": "Please close this ticket once the first trade has been placed",
                            "type": "text"
                            }
                        ]
                        }
                    ]
                },
                "assignee": {
                    "id": "5bfe63e1ec71bd223bbe623c"
                },
                "labels": [
                    "trades",
                    "newPortfolio"
                ]
            }
        }
    )
)

{'content': {'Content': {'fields': {'assignee': {'id': '5bfe63e1ec71bd223bbe623c'},
                                    'description': {'content': [{'content': [{'text': 'A '
                                                                                      'portfolio '
                                                                                      'has '
                                                                                      'been '
                                                                                      'created. '
                                                                                      'Please '
                                                                                      'make '
                                                                                      'your '
                                                                                      'first '
                                                              

In [10]:
subscriptions_api.delete_subscription(
    scope=subscription_scope,
    code=subscription_code
)

configuration_sets_api.delete_configuration_set(
    type="Shared",
    scope="FinbourneUniversity",
    code="jira"
)

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What have we covered?</p>

<p style="font-family: Montserrat; color: #203443">- We created a LUSID webhook notification using the Notifications API via the Python SDK.</p>

<p style="font-family: Montserrat; color: #203443">- We looked at the available content types, authentication methods and request types for webhook notifications.</p>

<p style="font-family: Montserrat; color: #203443">- We created a 3rd party webhook notification using the Notifications API via the Python SDK.</p>